In [1]:
# Deep Learning libs
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Flatten,Conv2D,Reshape,Dropout,MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import gc
#Loading the data from google drive 

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Creating a tensorflow dataset

In [0]:
def extract_fn_2(tfrecord):
        # Extract features using the keys set during creation
        features = {
            'feature1': tf.FixedLenFeature([128], tf.float32,),
            'feature2': tf.FixedLenFeature([128], tf.float32),
            'label': tf.FixedLenFeature([10],tf.int64)
        }

        # Extract the data record
        sample = tf.parse_single_example(tfrecord, features)
        data = tf.stack([sample['feature1'], sample['feature2']])
        label = sample['label']
        return [data,label] 
      

In [0]:
def extract_fn_4(tfrecord):
        # Extract features using the keys set during creation
        features = {
            'feature1': tf.FixedLenFeature([128], tf.float32),
            'feature2': tf.FixedLenFeature([128], tf.float32),
            'feature3': tf.FixedLenFeature([128], tf.float32),
            'feature4': tf.FixedLenFeature([128], tf.float32),
            'label': tf.FixedLenFeature([10],tf.int64)
        }

        # Extract the data record
        sample = tf.parse_single_example(tfrecord, features)
        data = tf.stack([sample['feature1'], sample['feature2'], sample['feature3'], sample['feature4']])
        label = sample['label']
        return [data,label] 

# Defining the Fully connected model
<ul>

1. The core data structure of Keras is a model. The simplest type of model is the Sequential model, a linear stack of layers.

2. A 3-layer deep neural network consisiting only of fully connected layers of size 512, 256, 11 neurons.
  

In [0]:
def build_cnn_model(in_shape):
  # Declare layers size
  conv1_kernel_shape=(3,1)
  conv1_number_of_filters=64
  conv2_kernel_shape=(3,2)
  conv2_number_of_filters=16
  dense1_size = 128
  dense2_size = 10
  dropout = 0.4

  # Build model
  model_conv = Sequential()
  model_conv.add(Reshape((128,in_shape[0],1), input_shape=in_shape))
  model_conv.add(Conv2D(conv1_number_of_filters, conv1_kernel_shape, strides=1,
                   padding='same', data_format='channels_last', activation='relu', kernel_initializer='he_normal'))
  model_conv.add(BatchNormalization())
  model_conv.add(MaxPooling2D())
  model_conv.add(Conv2D(conv2_number_of_filters, conv2_kernel_shape, strides=1,
                   padding='same', data_format='channels_last', activation='relu', kernel_initializer='he_normal'))
  model_conv.add(Flatten())
  model_conv.add(Dropout(rate=1-dropout))
  model_conv.add(Dense(dense1_size, activation='relu', kernel_initializer='he_normal'))
  model_conv.add(Dense(dense2_size, activation='softmax'))

  # Compile model
  model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_conv.summary()
  return model_conv

In [0]:
def train_cnn_model(train_dataset, val_dataset, input_shape, filename, epochs):
  model = build_cnn_model(input_shape)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
  mc = ModelCheckpoint(filename, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
  model.fit(train_dataset, epochs=epochs, steps_per_epoch=100,validation_steps=100, validation_data =val_dataset ,callbacks=[mc])

# Fitting the model

1. The training process will run for a fixed number of iterations (epochs) through the dataset.

2. We can also set the number of instances that are evaluated before a weight update in the network is performed (batch size).

## Model 1

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/int_train_data.tfrecord'])
train_dataset=train_dataset.map(extract_fn_2).repeat().batch(1000)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/int_val_data.tfrecord'])
val_dataset = val_dataset.map(extract_fn_2).repeat().batch(1000)

In [24]:
train_cnn_model(train_dataset,val_dataset,(2,128),'/content/drive/My Drive/int_model.h5',100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 128, 2, 1)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 2, 64)        256       
_________________________________________________________________
batch_normalization_v1_7 (Ba (None, 128, 2, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 1, 64)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 1, 16)         6160      
_________________________________________________________________
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
__________

## Model 2

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/der_train_data.tfrecord'])
train_dataset=train_dataset.map(extract_fn_2).repeat().batch(1000).prefetch(4)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/der_val_data.tfrecord'])
val_dataset = val_dataset.map(extract_fn_2).repeat().batch(1000).prefetch(4)

In [0]:
train_cnn_model(train_dataset,val_dataset,(2,128),'/content/drive/My Drive/BestModels/der_model.h5',100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_10 (Reshape)         (None, 128, 2, 1)         0         
_________________________________________________________________
batch_normalization_v1_20 (B (None, 128, 2, 1)         4         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 128, 2, 64)        256       
_________________________________________________________________
batch_normalization_v1_21 (B (None, 128, 2, 64)        256       
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 128, 2, 16)        6160      
_________________________________________________________________
dropout_10 (Dropout)         (None, 128, 2, 16)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4096)              0         
__________

## Model 3

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/raw_train_sklearn_split.tfrecord'])
train_dataset=train_dataset.map(extract_fn_2).repeat().batch(1000).prefetch(4)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/modulation_recognition_datasets/raw_val_sklearn_split.tfrecord'])
val_dataset = val_dataset.map(extract_fn_2).repeat().batch(1000).prefetch(4)

In [27]:
train_cnn_model(train_dataset,val_dataset,(2,128),'/content/drive/My Drive/Best_Models/raw_model.h5',100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 128, 2, 1)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 2, 64)        256       
_________________________________________________________________
batch_normalization_v1_8 (Ba (None, 128, 2, 64)        256       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 1, 64)         0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 1, 16)         6160      
_________________________________________________________________
flatten_7 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
__________

## Model 4

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com1_train_data.tfrecord'])
train_dataset=train_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com1_val_data.tfrecord'])
val_dataset = val_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)

In [29]:
train_cnn_model(train_dataset,val_dataset,(4,128),'/content/drive/My Drive/Best_Models/com1_model.h5',100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_8 (Reshape)          (None, 128, 4, 1)         0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 128, 4, 64)        256       
_________________________________________________________________
batch_normalization_v1_9 (Ba (None, 128, 4, 64)        256       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 64, 2, 64)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 64, 2, 16)         6160      
_________________________________________________________________
flatten_8 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 2048)              0         
__________

## Model 5

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com2_train_data.tfrecord'])
train_dataset=train_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com2_val_data.tfrecord'])
val_dataset = val_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)

In [31]:
train_cnn_model(train_dataset,val_dataset,(4,128),'/content/drive/My Drive/Best_Models/com2_model.h5',100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_9 (Reshape)          (None, 128, 4, 1)         0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 128, 4, 64)        256       
_________________________________________________________________
batch_normalization_v1_10 (B (None, 128, 4, 64)        256       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 64, 2, 64)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 2, 16)         6160      
_________________________________________________________________
flatten_9 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
__________

## Model 6

In [0]:
train_dataset=tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com3_train_data.tfrecord'])
train_dataset=train_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)
val_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/TFDatasets/com3_val_data.tfrecord'])
val_dataset = val_dataset.map(extract_fn_4).repeat().batch(1000).prefetch(4)

In [33]:
train_cnn_model(train_dataset,val_dataset,(4,128),'/content/drive/My Drive/Best_Models/com3_model.h5',150)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_10 (Reshape)         (None, 128, 4, 1)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 128, 4, 64)        256       
_________________________________________________________________
batch_normalization_v1_11 (B (None, 128, 4, 64)        256       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 2, 64)         0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 64, 2, 16)         6160      
_________________________________________________________________
flatten_10 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 2048)              0         
__________